In [6]:
import json
from tqdm import tqdm

In [2]:
# !wget https://huseinhouse-storage.s3-ap-southeast-1.amazonaws.com/bert-bahasa/dumping-twitter-6-july-2019.json
# !wget https://malaya-dataset.s3-ap-southeast-1.amazonaws.com/dumping/twitter/2020-02-22-twitter-dump-in.json
# !wget https://malaya-dataset.s3-ap-southeast-1.amazonaws.com/dumping/twitter/2020-03-08-twitter-dump-in.json
# !wget https://huseinhouse-storage.s3-ap-southeast-1.amazonaws.com/bert-bahasa/dumping-instagram-6-july-2019.json

In [3]:
with open('dumping-instagram-6-july-2019.json') as fopen:
    instagram = json.load(fopen)
    
len(instagram)

695571

In [4]:
with open('dumping-twitter-6-july-2019.json') as fopen:
    twitter = json.load(fopen)
    
len(twitter)

6597867

In [7]:
files = ['2020-02-22-twitter-dump-in.json', '2020-03-08-twitter-dump-in.json']

for file in files:
    print(file)

    with open(file) as fopen:
        temp = json.load(fopen)

    for i in tqdm(range(len(temp))):
        retweet = temp[i]['retweet_text_full']
        t = temp[i]['data_text']
        if retweet != 'NULL' and len(retweet) > len(t):
            t = retweet
        twitter.append(t)

2020-02-22-twitter-dump-in.json


100%|██████████| 1617795/1617795 [00:01<00:00, 1416202.37it/s]


2020-03-08-twitter-dump-in.json


100%|██████████| 1711555/1711555 [00:01<00:00, 1436409.10it/s]


In [8]:
import cleaning

In [9]:
twitter = twitter + instagram

In [10]:
import re

def preprocessing(string):
    string = re.sub(
        'http\S+|www.\S+',
        '',
        ' '.join(
            [i for i in string.split() if i.find('#') < 0 and i.find('@') < 0]
        ),
    )
    
    chars = ',.()!:\'"/;=-'
    for c in chars:
        string = string.replace(c, f' {c} ')
        
    string = re.sub(
        u'[0-9!@#$%^&*()_\-+{}|\~`\'";:?/.>,<]',
        ' ',
        string,
        flags = re.UNICODE,
    )
    string = re.sub(r'[ ]+', ' ', string).strip()
    
    return string.lower()

def loop(strings):
    for i in tqdm(range(len(strings))):
        strings[i] = preprocessing(strings[i])
    return strings

In [11]:
%%time

twitter = cleaning.multiprocessing(twitter, loop)

100%|██████████| 663924/663924 [00:46<00:00, 14262.37it/s]


CPU times: user 7.6 s, sys: 5.51 s, total: 13.1 s
Wall time: 57.3 s


In [12]:
%%time

temp_vocab = list(set(cleaning.multiprocessing(twitter, cleaning.unique_words)))

CPU times: user 4.48 s, sys: 2.62 s, total: 7.1 s
Wall time: 15.6 s


In [13]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.duplicate_dots_marks_exclamations, list_mode = False)
print(len(temp_dict))

0
CPU times: user 366 ms, sys: 871 ms, total: 1.24 s
Wall time: 2.73 s


In [14]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 663924/663924 [00:10<00:00, 61318.37it/s]


CPU times: user 5.89 s, sys: 4.61 s, total: 10.5 s
Wall time: 20.3 s


In [15]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.remove_underscore, list_mode = False)
print(len(temp_dict))

0
CPU times: user 364 ms, sys: 1.08 s, total: 1.45 s
Wall time: 1.84 s


In [16]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 663924/663924 [00:10<00:00, 61684.40it/s]


CPU times: user 5.83 s, sys: 5.09 s, total: 10.9 s
Wall time: 20.8 s


In [17]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.isolate_spamchars, list_mode = False)
print(len(temp_dict))

13
CPU times: user 335 ms, sys: 1e+03 ms, total: 1.33 s
Wall time: 1.7 s


In [18]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 663924/663924 [00:12<00:00, 52293.41it/s]


CPU times: user 5.8 s, sys: 4.91 s, total: 10.7 s
Wall time: 22.3 s


In [19]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.break_short_words, list_mode = False)
print(len(temp_dict))

0
CPU times: user 304 ms, sys: 1.15 s, total: 1.46 s
Wall time: 1.64 s


In [20]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 663924/663924 [00:13<00:00, 49062.70it/s]


CPU times: user 6.24 s, sys: 4.85 s, total: 11.1 s
Wall time: 23.9 s


In [21]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.break_long_words, list_mode = False)
print(len(temp_dict))

0
CPU times: user 318 ms, sys: 1.06 s, total: 1.38 s
Wall time: 1.62 s


In [22]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 663924/663924 [00:10<00:00, 61222.76it/s]


CPU times: user 5.85 s, sys: 4.88 s, total: 10.7 s
Wall time: 20.6 s


In [23]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.remove_ending_underscore, list_mode = False)
print(len(temp_dict))

0
CPU times: user 350 ms, sys: 1.19 s, total: 1.54 s
Wall time: 1.79 s


In [24]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 663924/663924 [00:10<00:00, 61667.02it/s]


CPU times: user 5.93 s, sys: 4.57 s, total: 10.5 s
Wall time: 20.5 s


In [25]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.remove_starting_underscore, list_mode = False)
print(len(temp_dict))

0
CPU times: user 329 ms, sys: 1.1 s, total: 1.43 s
Wall time: 1.63 s


In [26]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 663924/663924 [00:10<00:00, 61576.86it/s]


CPU times: user 6.07 s, sys: 4.75 s, total: 10.8 s
Wall time: 20.6 s


In [27]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.end_punct, list_mode = False)
print(len(temp_dict))

7786
CPU times: user 358 ms, sys: 1.21 s, total: 1.57 s
Wall time: 1.79 s


In [28]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 663924/663924 [00:11<00:00, 57084.48it/s]


CPU times: user 6.04 s, sys: 4.67 s, total: 10.7 s
Wall time: 21.6 s


In [29]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.start_punct, list_mode = False)
print(len(temp_dict))

8141
CPU times: user 361 ms, sys: 1.12 s, total: 1.48 s
Wall time: 1.68 s


In [30]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 663924/663924 [00:15<00:00, 42160.41it/s]


CPU times: user 6.69 s, sys: 4.61 s, total: 11.3 s
Wall time: 26.4 s


In [31]:
%%time
temp_dict = cleaning.multiprocessing(temp_vocab, cleaning.join_dashes, list_mode = False)
print(len(temp_dict))

0
CPU times: user 299 ms, sys: 1.18 s, total: 1.48 s
Wall time: 1.73 s


In [32]:
%%time

twitter = cleaning.multiprocessing_multiple(twitter, temp_dict, cleaning.string_dict_cleaning)

100%|██████████| 663924/663924 [00:11<00:00, 59864.54it/s]


CPU times: user 5.87 s, sys: 4.38 s, total: 10.2 s
Wall time: 20.6 s


In [33]:
twitter = [s.lower() for s in twitter]

In [34]:
twitter[:100]

['ternyata kl lg sdih bisa ngasilin makanan enak',
 'abu kampret',
 'bapa saya suka pake oppo saya sukanya nokia kaka saya sukanya samsung yg penting punya hape aja',
 'ngelamar kasih cincin tp kok mukanya songong ya sedih gue liatnya',
 'caption iki nggarai uwong males nikah min ya kali manusia arep punah ngunu neg gak nikah',
 'pertanyaannya sederhana jika kami memang dukung prabowo ngapain selama kampanye kemarin capek dukung jokowi sampa',
 '',
 'memiliki sedikit iman lebih berharga dari pada memiliki segudang emas',
 'untuk mengamankan suara partai ahmad rofiq selaku sekjen partai perindo meminta kepada seluruh caleg dan struktur',
 'dom jakpus sih bebas mau ketemuan or shopee',
 'bisa dapet duit ini kaga punya mobil juga kan kaya gemer gemer ini kaga',
 'on jadahnyaaa in sorry bad english hihuheheho',
 'valentino rossi tidak setuju kompetisi motogp dimulai dari eropa',
 'sis tak faham apa yang mungkin ini puncanya tu',
 'martabak terang bulan martabak untuk yg asin gurih a k a ma

In [35]:
with open('ms-socialmedia.json', 'w') as fopen:
    json.dump(twitter, fopen)

In [37]:
# !wget https://malaya-dataset.s3-ap-southeast-1.amazonaws.com/dumping/twitter/2020-02-22-twitter-dump-en.json
# !wget https://malaya-dataset.s3-ap-southeast-1.amazonaws.com/dumping/twitter/2020-03-08-twitter-dump-en.json

In [38]:
files = ['2020-02-22-twitter-dump-en.json', '2020-03-08-twitter-dump-en.json']

twitter_en = []

for file in files:
    print(file)

    with open(file) as fopen:
        temp = json.load(fopen)

    for i in tqdm(range(len(temp))):
        retweet = temp[i]['retweet_text_full']
        t = temp[i]['data_text']
        if retweet != 'NULL' and len(retweet) > len(t):
            t = retweet
        twitter_en.append(t)

2020-02-22-twitter-dump-en.json


100%|██████████| 717858/717858 [00:00<00:00, 1241059.53it/s]


2020-03-08-twitter-dump-en.json


100%|██████████| 716237/716237 [00:00<00:00, 1302148.73it/s]


In [39]:
%%time

twitter_en = cleaning.multiprocessing(twitter_en, loop)

100%|██████████| 89630/89630 [00:02<00:00, 42444.29it/s]


CPU times: user 1.11 s, sys: 1.65 s, total: 2.76 s
Wall time: 4.63 s


In [42]:
twitter_en = [s.lower() for s in twitter_en]
twitter_en = list(filter(None, twitter_en))

In [44]:
with open('en-socialmedia.json', 'w') as fopen:
    json.dump(twitter_en, fopen)